In [ ]:
https://umap-learn.readthedocs.io/en/latest/performance.html
https://pair-code.github.io/understanding-umap/    

In [1]:
import pickle
import numpy as np

import umap
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [2]:
file = open('../data/features_dict.pkl', "rb")
features_dict = pickle.load(file)


In [10]:
def features_preprocess(features_dict: dict, n_neighbors, min_dist):
    filenames = np.array(list(features_dict.keys()))
    feat = np.array(list(features_dict.values()))
    feat = feat.reshape(-1, 4096)
    # pca = PCA(n_components=100, random_state=22)
    # pca.fit(feat)
    # data_after_pca = pca.transform(feat)
    
#     standard_embedding = TSNE(n_components=2, learning_rate=learning_rate, init='random', random_state = 42, 
#                               perplexity = perplexity, early_exaggeration = early_exaggeration).fit_transform(feat)
    standard_embedding = umap.UMAP(random_state=42, n_neighbors = n_neighbors, min_dist = min_dist).fit_transform(feat)
#     print(standard_embedding)
    return pd.DataFrame({'Name': filenames, 'Vector1': standard_embedding[:, 0], 'Vector2': standard_embedding[:, 1]})

In [11]:
def prepare_info_df(df):
    df['Row'] = df['Name'].apply(lambda x: x[1:3]).apply(lambda x: chr(int(x) + 64))
    df['Column'] = df['Name'].apply(lambda x: x[4:6])
    df['F'] = df['Name'].apply(lambda x: x[7:9])
    df['Well'] = df['Row'] + df['Column']
    for channel_str in ['ch1', 'ch2', 'ch3', 'ch4']:  # here comes the problem
        df['Name'] = df['Name'].apply(lambda x: x.replace(channel_str, ''))
    df = df.drop_duplicates()
    well_df = pd.read_csv('E:\data\Cell-Painting-HepG2-Plate-Layouts.csv',
                          usecols=[0, 1, 2])
    df = df.merge(well_df)
    df.rename(columns={'Concentration [uM]': 'Concentration'}, inplace=True)
    return df

In [12]:
result = features_preprocess(features_dict, 15, 0.1)
result = prepare_info_df(result)
result

,Name,Vector1,Vector2,Row,Column,F,Well,Compound,Concentration
0,r01c01f01p01-sk1fk1fl1.tiff,0.811826,4.881280,A,01,01,A01,Berberine Chloride,50.0
1,r01c01f04p01-sk1fk1fl1.tiff,-2.097018,3.710042,A,01,04,A01,Berberine Chloride,50.0
2,r01c01f07p01-sk1fk1fl1.tiff,-0.173647,0.306713,A,01,07,A01,Berberine Chloride,50.0
3,r01c02f01p01-sk1fk1fl1.tiff,0.764021,4.751930,A,02,01,A02,Berberine Chloride,25.0
4,r01c02f04p01-sk1fk1fl1.tiff,-1.787187,3.228990,A,02,04,A02,Berberine Chloride,25.0
...,...,...,...,...,...,...,...,...,...
1147,r16c23f04p01-sk1fk1fl1.tiff,-0.032225,-0.085732,P,23,04,P23,DMSO,NaN
1148,r16c23f07p01-sk1fk1fl1.tiff,-1.919574,1.239270,P,23,07,P23,DMSO,NaN
1149,r16c24f01p01-sk1fk1fl1.tiff,-2.268742,2.377045,P,24,01,P24,DMSO,NaN
1150,r16c24f04p01-sk1fk1fl1.tiff,-2.382475,3.196472,P,24,04,P24,DMSO,NaN


In [14]:
def gen_fig(n_neighbors = 15, min_dist = 0.1):
    result = features_preprocess(features_dict, n_neighbors, min_dist)
    result = prepare_info_df(result)
    fig = px.scatter(result, x = 'Vector1', y = 'Vector2', color = 'Compound')
    return fig
gen_fig().show()

In [15]:
app = JupyterDash(__name__)
app.layout = html.Div([
    html.Div([ 
        dcc.Graph(id='graph', figure=gen_fig())
    ]),
    html.Div([
        'n_neighbors',
        dcc.Slider(
        id='n_neighbors',
        min=5,
        max=200,
        step=5,
        value=15,
        tooltip={"placement": "right",
                "always_visible": True},
    ),
    ]),
    html.Div([
        'min_dist',
        dcc.Slider(
        id='min_dist',
        min=0,
        max=0.95,
        step=0.05,
        value=0.1,
        tooltip={"placement": "right",
            "always_visible": True},
    ),
    ]),
    
    
])

In [16]:
@app.callback(
    Output('graph', 'figure'),
    [Input('n_neighbors', 'value'),
     Input('min_dist', 'value')])
def update_output(value1, value2):
    return gen_fig(n_neighbors = value1, min_dist = value2)

In [17]:
app.run_server( port = 8075, dev_tools_ui=True, #debug=True,
              dev_tools_hot_reload =True, threaded=True)

Dash app running on http://127.0.0.1:8075/


C:\Users\Krul\Anaconda3\lib\site-packages\jupyter_dash\jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.

